### Web page QnA Rag

Load html content as text

In [25]:
from langchain_community.document_loaders import WebBaseLoader
page_url = "https://learn.microsoft.com/en-us/azure/storage/common/storage-account-overview"
loader = WebBaseLoader(web_paths=[page_url])
content  = loader.load()
LLM_MODEL_NAME = "llama3:8b"
EMBEDDING_MODEL_NAME="nomic-embed-text"

Document chunking

In [26]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=80)
texts = text_splitter.split_documents(content)

Vector store initialization & similarity search

In [27]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
embedding_db = OllamaEmbeddings(model=EMBEDDING_MODEL_NAME)
fdb = FAISS.from_documents(documents=texts , embedding=embedding_db)
fdb.similarity_search_with_score(query="Account types?")


[(Document(id='1af2ae71-f41f-45f2-adfa-43f1fff6daf0', metadata={'source': 'https://learn.microsoft.com/en-us/azure/storage/common/storage-account-overview', 'title': 'Storage account overview - Azure Storage | Microsoft Learn', 'description': 'Learn about the different types of storage accounts in Azure Storage. Review account naming, performance tiers, access tiers, redundancy, encryption, endpoints, and more.', 'language': 'en-us'}, page_content="Storage account overview\n\n\n\n\n\n\t\t\t\t2025-03-04\n\t\t\t\n\n\n\n\n\n\n\n\nFeedback\n\n\n\n\n\n\t\t\t\tIn this article\n\t\t\t\n\nAn Azure storage account contains all of your Azure Storage data objects: blobs, files, queues, and tables. The storage account provides a unique namespace for your Azure Storage data that's accessible from anywhere in the world over HTTP or HTTPS. Data in your storage account is durable and highly available, secure, and massively scalable.\nTo learn how to create an Azure Storage account, see Create a storag

##### With retrieval chain (Get response from llm)

First create document chain (combined context)

`create_stuff_documents_chain` -> combines docs in a single context which in inputted to retreiver


In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_ollama import ChatOllama

llm_model = ChatOllama(model=LLM_MODEL_NAME)

prompt_template = ChatPromptTemplate.from_template(
    """
        Answer the following question based only on the provided context:
            <context>
            {context}
            </context>
    """
)

document_chain = create_stuff_documents_chain(llm_model, prompt_template)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provided context:\n            <context>\n            {context}\n            </context>\n    '), additional_kwargs={})])
| ChatOllama(model='llama3:8b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

`create_retrieval_chain` -> uses the vector store retreiver to pull necessry context from the document chain defined earlier and forward only the relevant documents to llm.

Relevant documents are fetched with similarity search

In [29]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(fdb.as_retriever(), document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017A7A0290C0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provided context:\n            <context>\n            {context}\n            </context>\n  

In [30]:
resp = retrieval_chain.invoke({"input":"How to Transfer data into a storage account?"})
resp

{'input': 'How to Transfer data into a storage account?',
 'context': [Document(id='6af4494f-83be-4340-9392-d43d98ffc5bc', metadata={'source': 'https://learn.microsoft.com/en-us/azure/storage/common/storage-account-overview', 'title': 'Storage account overview - Azure Storage | Microsoft Learn', 'description': 'Learn about the different types of storage accounts in Azure Storage. Review account naming, performance tiers, access tiers, redundancy, encryption, endpoints, and more.', 'language': 'en-us'}, page_content="Transfer data into a storage account\nMicrosoft provides services and utilities for importing your data from on-premises storage devices or third-party cloud storage providers. Which solution you use depends on the quantity of data you're transferring. For more information, see Azure Storage migration overview.\nStorage account encryption\nAll data in your storage account is automatically encrypted on the service side. For more information about encryption and key managemen